<a href="https://colab.research.google.com/github/HarishSingh1981/LearningNeuralNetwork/blob/main/Assign_5_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [11]:
#attempt 3
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        #input 1x28x28 -? OUtput 10x26x26? RF 3
        self.convblk1 = nn.Sequential(nn.Conv2d(1, 10, 3),
                                      nn.BatchNorm2d(10),
                                      nn.ReLU())
                                      #nn.Dropout2d(0.5))
        #input 10x26x26 -? OUtput 10x24x24? RF 5
        self.convblk2 = nn.Sequential(nn.Conv2d(10, 10, 3),
                                      nn.BatchNorm2d(10),
                                      nn.ReLU())
                                      #nn.Dropout2d(0.5))
        #input 10x24x24 -? OUtput 10x12x12? RF 6
        self.transitionBlock1 = nn.MaxPool2d(2,2)

        #input 10x12x12 -? OUtput 20x10x10? RF 10
        self.convblk3 = nn.Sequential(nn.Conv2d(10,32,3),
                                      nn.BatchNorm2d(32),
                                      nn.ReLU())
                                      #nn.Dropout2d(0.5))

        #input 20x10x10 -? OUtput 10x10x10? RF 14
        self.convblk4 = nn.Sequential(nn.Conv2d(32,10,3,padding=1),
                                      nn.BatchNorm2d(10),
                                      nn.ReLU())
                                      #nn.Dropout2d(0.5))
        #input 10x10x10 -? OUtput 10x8x8? RF 16
        self.convblk5 = nn.Sequential(nn.Conv2d(10,10,3),
                                      nn.BatchNorm2d(10),
                                      nn.ReLU())
                                      #nn.Dropout2d(0.5))
        #input 10x8x8 -? OUtput 20x6x6? RF 18
        self.convblk6 = nn.Sequential(nn.Conv2d(10,20,3),
                                      nn.BatchNorm2d(20),
                                      nn.ReLU())
                                      #nn.Dropout2d(0.5))
        #input 20x6x6 -? OUtput 10x6x6? RF 20
        self.convblk7 = nn.Sequential(nn.Conv2d(20,10,1),
                                      nn.BatchNorm2d(10),
                                      nn.ReLU())
                                      #nn.Dropout2d(0.5))
       
        #input 10x5x5 -? OUtput 10x1x1? RF 33
        self.avgPoolblk = nn.AvgPool2d(6,6)
    def forward(self, x):
      #maxpool must be used at least after 2 convolution and sud be as far as possible from last layer
        x = self.convblk1(x)
        x = self.convblk2(x)
        x = self.convblk3(x)
        x = self.transitionBlock1(x)
        x = self.convblk4(x)
#        x = self.transitionBlock2(x)
        x = self.convblk5(x)
        x = self.convblk6(x)
        x = self.convblk7(x)
        x = self.avgPoolblk(x)
        #x = self.conv8(x)
        #relu must not be used in last layer. The information must be intact in last layer
        #x = F.relu(self.conv7(x))
        #x = self.conv7(x)
        #x = self.gap(x)
        x = x.view(-1, 10)
        return F.log_softmax(x)

In [12]:
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 10, 26, 26]             100
       BatchNorm2d-2           [-1, 10, 26, 26]              20
              ReLU-3           [-1, 10, 26, 26]               0
            Conv2d-4           [-1, 10, 24, 24]             910
       BatchNorm2d-5           [-1, 10, 24, 24]              20
              ReLU-6           [-1, 10, 24, 24]               0
            Conv2d-7           [-1, 32, 22, 22]           2,912
       BatchNorm2d-8           [-1, 32, 22, 22]              64
              ReLU-9           [-1, 32, 22, 22]               0
        MaxPool2d-10           [-1, 32, 11, 11]               0
           Conv2d-11           [-1, 10, 11, 11]           2,890
      BatchNorm2d-12           [-1, 10, 11, 11]              20
    

<ipython-input-11-44ff7539039b>:65: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)


In [13]:
import torchvision


torch.manual_seed(1)
batch_size = 128

#kwargs = {'num_workers': 4, 'pin_memory': True} if use_cuda else {}
kwargs = dict(num_workers= 4, pin_memory= True if use_cuda else False,shuffle=True,batch_size=batch_size)

train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,)),
                        transforms.RandomRotation(degrees=7)
                    ])),**kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                        #transforms.RandomRotation(degrees=7)
                    ])),**kwargs)


In [14]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    train_loss = 0
    correct = 0
    processed=0
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        train_loss +=loss
        pred = output.argmax(dim=1,keepdim=True)
        #need to understand what does it do
        correct += pred.eq(target.view_as(pred)).sum().item()
        processed += len(data)
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx} Accurary={100*correct/processed:0.2f}')

    #train_loss /= len(train_loader.dataset)
   # print('\nTrain set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
     #   train_loss, correct, len(test_loader.dataset),
      #  100. * correct / len(train_loader.dataset)))
def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [15]:
from torch.optim.lr_scheduler import StepLR
model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.1, momentum=0.9)
for epoch in range(15):
    print(f'Epoch--> {epoch+1}')
    train(model, device, train_loader, optimizer, epoch)
    print("Current learning rate:", optimizer.param_groups[0]['lr'])
    test(model, device, test_loader)

Epoch--> 1


  0%|          | 0/469 [00:00<?, ?it/s]<ipython-input-11-44ff7539039b>:65: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)
loss=0.09933352470397949 batch_id=468 Accurary=93.91: 100%|██████████| 469/469 [01:44<00:00,  4.50it/s]


Current learning rate: 0.1

Test set: Average loss: 0.0819, Accuracy: 9754/10000 (97.54%)

Epoch--> 2


loss=0.010804458521306515 batch_id=468 Accurary=98.03: 100%|██████████| 469/469 [01:42<00:00,  4.56it/s]


Current learning rate: 0.1

Test set: Average loss: 0.0548, Accuracy: 9834/10000 (98.34%)

Epoch--> 3


...

Current learning rate: 0.1



Test set: Average loss: 0.0205, Accuracy: 9935/10000 (99.35%)

Epoch--> 15


loss=0.07098787277936935 batch_id=468 Accurary=99.29: 100%|██████████| 469/469 [01:38<00:00,  4.75it/s]

Current learning rate: 0.1



Test set: Average loss: 0.0212, Accuracy: 9935/10000 (99.35%)

